Imports

In [8]:
import os
import json
import requests
from bs4 import BeautifulSoup

Utils


In [9]:
def extract(ancestor, selector, attribute=None, multiple=False):
    if selector: 
        if multiple:
            if attribute:
                return [tag[attribute] for tag in ancestor.select(selector)]
            return [tag.get_text() for tag in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute]
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text()
        except AttributeError:
            return None
    try: 
        return ancestor[attribute]
    except (TypeError, KeyError):
        return None


Extraction of opinions 

In [10]:
selectors = {
    "opinion_id": (None, "data-entry-id"),
    "author": ("span.user-post__author-name",),
    "recommendation":  ("span.user-post__author-recomendation > em"),
    "stars": ("span.user-post__score-count"),
    "content": ("div.user-post__text"),
    "pros": ("div.review-feature__item--positive", None, True),
    "cons": ("div.review-feature__item--negative", None, True),
    "vote_yes": ("button.vote-yes","data-total-vote"),
    "vote_no":  ("button.vote-no","data-total-vote"),
    "published": ("span.user-post__published > time:nth-child(1)","datetime"),
    "purchased": ("span.user-post__published > time:nth-child(2)","datetime"),
}

In [11]:
with open("./cookie.json", "r") as jf:
    headers = json.load(jf)

In [13]:
product_id = "84514582"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while next_page:
    response = requests.get(next_page, headers = headers)
    if response.status_code == 200:
        print(next_page)
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(opinions))
        for opinion in opinions:
            single_opinion = {
                key: extract(opinion, *value)
                for key, value in selectors.items()
            }
            
            all_opinions.append(single_opinion)
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            next_page = None

https://www.ceneo.pl/84514582#tab=reviews
10


TypeError: extract() takes from 2 to 4 positional arguments but 42 were given

In [5]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, ensure_ascii=False, indent=4)